In [1]:
import warnings
warnings.filterwarnings('ignore')
import pyfolio as pf
import pandas as pd
import numpy as np
from leveraged_meta_lib import leveraged_meta
import common_perf_ana as cpa
import datetime

# %pip uninstall yfinance // we have to uninstall the old version and install the new (2023-01) version
# %pip install yfinance
# %pip install nbformat
# %pip install plotly
# %pip install pandas-market-calendars

In [2]:

meta_parameters = {
    'meta_rebalance_unit' : 'Month',
    'meta_rebalance_freq' : 1,
    'meta_rebalance_shift' : 0,
    'meta_lb_period' : 3,
    'meta_skipped_period' : 0,
    'meta_fixed_substrat_weights' : {'DualMom' : 2, 'BAA_AggDef' : 2, 'BAA_BalDef' : 2, 'TAA' : 2, 'KellerProtMom' : 2, 'NovellTactBond' : 1, 'HAA' : 1},
    'meta_rel_mom_substrat_weights' : [1, 2, 3, 4, 5, 6, 7], #[Top Perfromer, Second Perf., Third Perf., Fourth Perf., Fifth Perf., Sixth Perf., Seventh Perf.]
    'meta_sharpe_substrat_weights' : [1, 2, 3, 4, 5, 6, 7], #[Top Perfromer, Second Perf., Third Perf., Fourth Perf., Fifth Perf., Sixth Perf., Seventh Perf.]
    'meta_sortino_substrat_weights' : [1, 2, 3, 4, 5, 6, 7], #[Top Perfromer, Second Perf., Third Perf., Fourth Perf., Fifth Perf., Sixth Perf., Seventh Perf.]
    'meta_rel_mom_abs_threshold' : -1,
    'meta_sharpe_abs_threshold' : -100,
    'meta_sortino_abs_threshold' : -100,
    'meta_start_date' : '2004-01-01',
    'meta_end_date' : datetime.date.today(),
    'used_substrat_weights' : 'fixed_based' # 'fixed_based', 'ew_based', 'rel_mom_based', 'sharpe_based', 'sortino_based' 
}

meta_leverage_parameters = {
    # 'meta_overall_leverage' : 1,
    # 'meta_ETF_perf_leverage_threshold' : [0.2, 0.4, 0.6, 0.8, 1],
    # 'meta_ETF_perf_leverage' : [1, 1, 1, 1, 1],
    # 'meta_monthly_seas_based_leverage' : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    # 'meta_leverage_lookback_years': 5,
    # 'meta_tlt_based_leverage' : [1, 1, 1, 1] #[Bull, Rebound, Correction, Bear]
    # 'meta_overall_leverage' : 1,
    # 'meta_ETF_perf_leverage_threshold' : [0.2, 0.4, 0.6, 0.8, 1],
    # 'meta_ETF_perf_leverage' : [2, 1.5, 1, 0.75, 0.5],
    # 'meta_monthly_seas_based_leverage' : [1.5, 1.5, 1.25, 1.25, 1, 1, 1, 1, 1, 1, 1, 1],
    # 'meta_leverage_lookback_years': 5,
    # 'meta_tlt_based_leverage' : [1.1, 1.1, 1, 0.8] #[Bull, Rebound, Correction, Bear]
    'meta_overall_leverage' : 1.5,
    'meta_ETF_perf_leverage_threshold' : [0.2, 0.4, 0.6, 0.8, 1],
    'meta_ETF_perf_leverage' : [1.5, 1.25, 1, 0.85, 0.7],
    # 'meta_ETF_perf_leverage' : [1, 1, 1, 1, 1],
    'meta_monthly_seas_based_leverage' : [1.5, 1.5, 1.25, 1.25, 1, 1, 1, 1, 1, 1, 0.85, 0.85],
    # 'meta_monthly_seas_based_leverage' : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'meta_leverage_lookback_years': 5,
    'meta_tlt_based_leverage' : [1.25, 1.4, 1, 0.75] #[Bull, Rebound, Correction, Bear]
}

taa_parameters = {
    'taa_ticker_list' : ['VNQ', 'EEM', 'DBC', 'SPY', 'QQQ', 'TLT'],
    'taa_perc_ch_lb_list' : [60, 120, 180, 250],
    'taa_vol_lb' : 20,
    'taa_perc_ch_up_thres' : 0.75,
    'taa_perc_ch_low_thres' : 0.25,
    'taa_rebalance_unit' : 'Month',
    'taa_rebalance_freq' : 1,
    'taa_rebalance_shift' : 0
}

bold_parameters = {
    'baa_ticker_list_canary' : ['SPY', 'EEM', 'EFA', 'AGG'],
    'baa_ticker_list_defensive' : ['TIP', 'DBC', 'BIL', 'IEF', 'TLT', 'LQD', 'AGG'],
    # 'baa_ticker_list_aggressive' : ['QQQ'],
    'baa_ticker_list_aggressive' : ['QQQ', 'EEM', 'EFA', 'AGG'],
    'baa_ticker_list_balanced' : ['SPY', 'QQQ', 'IWM', 'VGK', 'EWJ', 'EEM', 'VNQ', 'DBC', 'GLD', 'TLT', 'HYG', 'LQD'],
    'baa_rebalance_unit' : 'Month',
    'baa_rebalance_freq' : 1,
    'baa_rebalance_shift' : 0,
    'baa_skipped_period' : 0,
    'baa_no_played_ETFs' : {'offAgg' : 1, 'offBal' : 6, 'def' : 3},
    'baa_abs_threshold' : -1
}

dual_mom_parameters = {
    'dm_tickers_list' : ['VNQ', 'EEM', 'DBC', 'SPY', 'TLT', 'SHY'],
    'dm_rebalance_unit' : 'Month',
    'dm_rebalance_freq' : 1,
    'dm_rebalance_shift' : 0,
    'dm_lb_period' : 4,
    'dm_skipped_period' : 0,
    'dm_no_played_ETFs' : 3,
    'dm_sub_rank_weights' : {'relMom' : 0.5, 'volatility' : 0.25, 'correlation' : 0.25},
    'dm_abs_threshold' : 0
}

keller_protmom_parameters = {
    'protmom_tickers_list' : ['SPY', 'QQQ', 'IWM', 'VNQ', 'DBC', 'GLD', 'HYG', 'LQD', 'TLT'],
    'protmom_rebalance_unit' : 'Month',
    'protmom_rebalance_freq' : 1,
    'protmom_rebalance_shift' : 0,
    'protmom_correl_lb_months' : 12,
    'protmom_lb_periods' : [1, 3, 6, 12],
    'protmom_lb_weights' : [1, 1, 1, 1],
    'protmom_selected_ETFs' : 3
}

novell_tactbond_parameters = {
    'tactbond_tickers_list' : ['BNDX', 'EMB', 'HYG', 'IEF', 'LQD', 'SHY', 'TIP', 'TLT'],
    'tactbond_rebalance_unit' : 'Month',
    'tactbond_rebalance_freq' : 1,
    'tactbond_rebalance_shift' : 0,
    'tactbond_absolute_threshold' : 0,
    'tactbond_threshold_type' : 1, # 1 - Relative threshold to BIL, 0 - Absolute threshold
    'tactbond_cash_subs' : 1, # 1 - BIL, 0 - Cash
    'tactbond_lb_periods' : [1, 3, 6, 12],
    'tactbond_lb_weights' : [0, 0, 1, 0],
    'tactbond_selected_ETFs' : 3
}

hybrid_aa_parameters = {
    'haa_ticker_list_canary' : ['TIP'],
    'haa_ticker_list_defensive' : ['BIL', 'IEF'],
    'haa_ticker_list_offensive' : ['SPY', 'IWM', 'VWO', 'VEA', 'VNQ', 'DBC', 'IEF', 'TLT'],
    'haa_rebalance_unit' : 'Month',
    'haa_rebalance_freq' : 1,
    'haa_rebalance_shift' : 0,
    'haa_skipped_period' : 0,
    'haa_no_played_ETFs' : {'off' : 4, 'def' : 1},
    'haa_abs_threshold' : 0
}

pv_played_fin, strat_played_rets, strat_played_weights, pos_played_fin, cash_played_fin, strat_played_curr_weights, strat_unlev_curr_weights, ETF_curr_levs, monthly_curr_lev, tlt_curr_lev = leveraged_meta(meta_parameters, taa_parameters, bold_parameters, dual_mom_parameters, keller_protmom_parameters, novell_tactbond_parameters, meta_leverage_parameters, hybrid_aa_parameters)
pd.set_option('display.float_format', '{:.2f}%'.format)
print('Unleveraged weights: ')
display(strat_unlev_curr_weights * 100)
print('Leveraged weights: ')
display(strat_played_curr_weights * 100)
print('Used overall leverage: ', '{:.2f}%'.format(meta_leverage_parameters['meta_overall_leverage'] * 100))
print('Used TLT based leverage: ', '{:.2f}%'.format(tlt_curr_lev * 100))
print('Used monthly performance based leverage: ', '{:.2f}%'.format(monthly_curr_lev * 100))
print('Used ETF performance based leverage: ')
display(ETF_curr_levs * 100)
cpa.performance_indicators(pv_played_fin)


[*********************100%***********************]  6 of 6 completed
[*********************100%***********************]  17 of 17 completed
[*********************100%***********************]  17 of 17 completed
[*********************100%***********************]  6 of 6 completed
[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  9 of 9 completed
[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  22 of 22 completed
Unleveraged weights: 


AGG    11.11%
BIL    19.44%
BNDX    0.00%
DBC     0.00%
EEM     7.64%
EFA     0.00%
EMB     4.17%
EWJ     0.00%
GLD     0.00%
HYG     4.17%
IEF     0.00%
IWM     0.00%
LQD    11.11%
QQQ     1.73%
SHY     0.00%
SPY    10.14%
TIP     0.00%
TLT     5.56%
VEA     0.00%
VGK     0.00%
VNQ     1.92%
VWO     0.00%
cash   23.02%
Name: 2023-03-08 00:00:00-05:00, dtype: float64

Leveraged weights: 


AGG    18.75%
BIL    32.81%
BNDX    0.00%
DBC     0.00%
EEM     7.30%
EFA     0.00%
EMB     5.86%
EWJ     0.00%
GLD     0.00%
HYG     5.86%
IEF     0.00%
IWM     0.00%
LQD    12.50%
QQQ     2.43%
SHY     0.00%
SPY    14.26%
TIP     0.00%
TLT     5.31%
VEA     0.00%
VGK     0.00%
VNQ     2.16%
VWO     0.00%
cash   -7.24%
Name: 2023-03-08 00:00:00-05:00, dtype: float64

Used overall leverage:  150.00%
Used TLT based leverage:  75.00%
Used monthly performance based leverage:  100.00%
Used ETF performance based leverage: 


AGG    150.00%
BIL    150.00%
BNDX   150.00%
DBC    125.00%
EEM     85.00%
EFA     70.00%
EMB    125.00%
EWJ     70.00%
GLD     85.00%
HYG    125.00%
IEF    150.00%
IWM    100.00%
LQD    100.00%
QQQ    125.00%
SHY    150.00%
SPY    125.00%
TIP     70.00%
TLT     85.00%
VEA     70.00%
VGK     85.00%
VNQ    100.00%
VWO    100.00%
cash    85.00%
Name: 2023-03-08 00:00:00-05:00, dtype: float64


Performance indicators of the strategy:
Total return:  4046.80%
CAGR:  21.43%
Annualized Mean Return:  20.69%
Annualized Standard Deviation:  15.72%
Sharpe ratio:  1.316
Maximum drawdown:  17.88%
Mar ratio:  1.198

